## All the import

In [2]:
import keras
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.normalization import BatchNormalization
from keras.layers.advanced_activations import PReLU
from keras.utils import np_utils, generic_utils
from keras.optimizers import SGD, RMSprop, Adadelta, Adagrad, Adam
from keras.layers.recurrent import LSTM
from keras.layers.embeddings import Embedding
from keras.layers.convolutional import Conv2D, MaxPooling2D,Conv1D, MaxPooling1D
from keras.models import model_from_config
from keras import regularizers
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.constraints import maxnorm
from keras.models import load_model
from keras.layers.convolutional import AveragePooling2D
from keras.callbacks import EarlyStopping


from sklearn import svm, grid_search
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.cross_validation import train_test_split
from sklearn.calibration import CalibratedClassifierCV
from sklearn.cross_validation import StratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_curve, auc
import numpy as np

import random
import gzip
from sklearn import svm
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.decomposition import PCA
from sklearn import metrics
from sklearn.metrics import roc_auc_score
from sklearn.cross_validation import train_test_split
from sklearn.grid_search import GridSearchCV
from sklearn.externals import joblib 
from scipy import sparse
import pdb
from math import  sqrt
from sklearn.metrics import roc_curve, auc
import theano
import subprocess as sp
import scipy.stats as stats
import argparse
import pandas as pd
import csv
from random import shuffle
from keras.layers import Merge

## Load the data

In [3]:
def clean(seq):
    seq = seq.split('\n')
    seq2 = ''
    for j in seq:
        seq2 = seq2 + j
    seq2 = seq2[2:len(seq2)-1]
    seq2 = seq2.split(' ')
    #print(seq2)
    seq3=[]
    for j in seq2:
        #print(j)
        if j=='':
            a=0
        else:
            seq3.append(float(j))
    return seq3

In [4]:
def load_data(nb_train = 165000,nb_val = 6000,nb_test = 5000):
    # load your data using this function
    verif = []

    verif = pd.read_csv("genomes/negatifs_m-m-str.csv", sep = "\t",header=None)
    verif = np.array(verif)
    print(len(verif))
    for i in range(len(verif)):
        for j in range(2):
            if len(verif[i,j]) <= 15:
                print(i,"erreur")
            

    neg1 = verif

    verif=[]

    verif = pd.read_csv("genomes/negatifs_divers-str.csv", sep = "\t",header=None)
    verif = np.array(verif)
    print(len(verif))
    for i in range(len(verif)):
        for j in range(2):
            if len(verif[i,j]) <= 15:
                print(i,"erreur")
    neg2 = verif
    verif=[]
    verif = pd.read_csv("genomes/negatifs_mouse_divers-str.csv", sep = "\t",header=None)
    verif = np.array(verif)
    print(len(verif))
    for i in range(len(verif)):
        for j in range(2):
            if len(verif[i,j]) <= 15:
                print(i,"erreur")
    neg3 = verif
    verif=[]

    verif = pd.read_csv("genomes/positifs_m-m-str.csv", sep = "\t",header=None)  
    verif = np.array(verif)
    print(len(verif))
    for i in range(len(verif)):
        for j in range(2):
            if len(verif[i,j]) <= 15:
                print(i,"erreur")
            
    pos1 = verif
    l = len(pos1)
    verif=[]

    verif = pd.read_csv("genomes/positifs_divers-str.csv", sep = "\t",header=None)  
    verif = np.array(verif)
    print(len(verif))
    for i in range(len(verif)):
        for j in range(2):
            if len(verif[i,j]) <= 15:
                print(i,"erreur")
            
    pos2 = verif
    l=l+len(pos2)
    verif=[]

    verif = pd.read_csv("genomes/positifs_mouse_divers-str.csv", sep = "\t",header=None)  
    verif = np.array(verif)
    print(len(verif))
    for i in range(len(verif)):
        for j in range(2):
            if len(verif[i,j]) <= 15:
                print(i,"erreur")
            
    pos3 = verif
    l=l+len(pos3)
    verif=[]

    bdd = np.concatenate((pos1,pos2,pos3,neg1,neg2,neg3))

    pos1 = []
    neg1 = []
    pos2 = []
    neg2 = []

    labels = np.zeros((len(bdd),1))

    bdd = np.concatenate((bdd,labels),axis=1)
    for i in range(l):
        bdd[i,4]=1
    labels=[]
    
    # shuffle total

    indices = np.arange(len(bdd))
    shuffle(indices)
    bdd = bdd[indices]
    indices=[]

    # 
    l = len(bdd)
    matrice = np.zeros((l,36,36,1)) #4
    matrice1 = np.zeros((l,36,36,1))
    matrice2 = np.zeros((l,36,4))
    matrice3 = np.zeros((l,36,4))
    matrice4 = np.zeros((l,36,4))
    matrice5 = np.zeros((l,36,4))

    for i in range(l):
        seq1 = bdd[i,0]
        seq2 = bdd[i,1]
        prob1 = np.array(clean(bdd[i,2]))
        prob2 = np.array(clean(bdd[i,3]))
        for j in range(len(seq1)):
            for k in range(len(seq2)):
                if (seq1[j]=='a' and seq2[k]=='u') or (seq1[j]=='u' and seq2[k]=='a'):
                    matrice[i,j,k,0] = 1
                    #matrice[i,j,k,1] = prob1[j]+prob2[k]
                elif (seq1[j]=='g' and seq2[k]=='c') or (seq1[j]=='c' and seq2[k]=='g'):
                    matrice[i,j,k,0] = 1
                    #matrice[i,j,k,1] = prob1[j]+prob2[k]
                elif (seq1[j]=='g' and seq2[k]=='u') or (seq1[j]=='u' and seq2[k]=='g'):
                    matrice[i,j,k,0] = 1
                    #matrice[i,j,k,1] = prob1[j]+prob2[k]
                matrice1[i,j,k,0] = prob1[j]+prob2[k]
        for j in range(len(seq1)):
            if seq1[j]=='a':
                matrice2[i,j,0] = 1
            elif seq1[j]=='u':
                matrice2[i,j,1] = 1
            elif seq1[j]=='g':
                matrice2[i,j,2] = 1
            elif seq1[j]=='c':
                matrice2[i,j,3] = 1
        for j in range(len(seq2)):
            if seq2[j]=='a':
                matrice3[i,j,0] = 1
            elif seq2[j]=='u':
                matrice3[i,j,1] = 1
            elif seq2[j]=='g':
                matrice3[i,j,2] = 1
            elif seq2[j]=='c':
                matrice3[i,j,3] = 1

    for i in range(36):
        matrice4[:,36-i-1,:] = matrice2[:,i,:]
    for i in range(36):
        matrice5[:,36-i-1,:] = matrice3[:,i,:]
    
    training = []
    training.append(matrice[:nb_train])
    training.append(matrice1[:nb_train])
    training.append(matrice2[:nb_train])
    training.append(matrice4[:nb_train])
    training.append(matrice3[:nb_train])
    training.append(matrice5[:nb_train])    
    
    validation = []
    validation.append(matrice[nb_train:nb_train+nb_val])
    validation.append(matrice1[nb_train:nb_train+nb_val])
    validation.append(matrice2[nb_train:nb_train+nb_val])
    validation.append(matrice4[nb_train:nb_train+nb_val])
    validation.append(matrice3[nb_train:nb_train+nb_val])
    validation.append(matrice5[nb_train:nb_train+nb_val])

    testing = []
    testing.append(matrice[-nb_test:])
    testing.append(matrice1[-nb_test:])
    testing.append(matrice2[-nb_test:])
    testing.append(matrice4[-nb_test:])
    testing.append(matrice3[-nb_test:])
    testing.append(matrice5[-nb_test:])

    labels = bdd[:,4]
    bdd = []
    y = labels[:nb_train]
    y = keras.utils.np_utils.to_categorical(y,2)
    val_y = labels[nb_train:nb_train+nb_val]
    val_y = keras.utils.np_utils.to_categorical(val_y,2)
    true_y = labels[-nb_test:]
    argtest=[]
    np.sum(y[:,1])
    return training, y, validation, val_y, testing, true_y

## Calculate the performance

In [5]:
def calculate_performance(test_num, pred_y,  labels):
    tp = 0
    fp = 0
    tn = 0
    fn = 0
    for index in range(test_num):
        if labels[index] ==1:
            if labels[index] == pred_y[index]:
                tp = tp +1
            else:
                fn = fn + 1
        else:
            if labels[index] == pred_y[index]:
                tn = tn +1
            else:
                fp = fp + 1               
            
    acc = float(tp + tn)/test_num
    precision = float(tp)/(tp+fp)
    sensitivity = float(tp)/(tp+fn)
    specificity = float(tn)/(tn+fp)
    MCC = float(tp*tn-fp*fn)/(np.sqrt((tp+fp)*(tp+fn)*(tn+fp)*(tn+fn)))
    return acc, precision, sensitivity, specificity, MCC 

In [6]:
from keras.layers.merge import concatenate

## Train and Evaluate the model

In [7]:
Data = load_data(100,250,20000)

26023
24229
37991
26023
24229
37993


In [8]:
def perfs_test(model,test,true):
    print('predicting')
            
    predictions = model.predict(test)[:,1]
    print(predictions)
    for i,nulll in enumerate(predictions):
        predictions[i] = round(predictions[i])
    print(predictions,true)
    perfs = calculate_performance(len(predictions), predictions, true)
    
    print("acc : ", perfs[0])
    print("precision : ", perfs[1])
    print("sensitivity : ", perfs[2])
    print("specificity : ", perfs[3])
    print("MCC : ", perfs[4])

## Run

## Récupération

In [9]:
# paramètres 
true_y = Data[5]
training = Data[0]
y = Data[1]
batch_size=64
epochs=25
verbose1 = 1
verbose2 = 1
validation = Data[2]
val_y = Data[3]
testing = Data[4]
matrixsize11 = 6
nbfilter11 = 24
matrixsize12 = 7
nbfilter12 = 4
matrixsize21 = 6
nbfilter21 = 24
matrixsize22 = 7
nbfilter22 = 4
nbfilter1 = 96
kernelsize = 7
nbfilters2 = 96
kernel_size2 = 7
Dense1 = 128
Dense2 = 512
Dense3 = 512
Dense4 = 128
Dense5 = 64

from keras.layers import merge
from keras import Model
k = matrixsize11
# init_weights
I = np.eye(k)
M = np.diag(np.ones(k-1),1) + np.diag(np.ones(k-1),-1) + np.eye(k)
I2 = np.zeros((k,k))
M2 = np.zeros((k,k))
for j in range(k):
    I2[:,j] = I[:,k-j-1]
    M2[:,j] = M[:,k-j-1]        
W = np.zeros((k,k,1,nbfilter11))
W[:,:,0,0] = I
W[:,:,0,1] = I2
W[:,:,0,2] = M
W[:,:,0,3] = M2
for j in range(4,nbfilter11):
    W[:,:,0,j] = np.random.randn(k,k)*0.2


k2 = matrixsize12
I = np.eye(k2)
M = np.diag(np.ones(k2-1),1) + np.diag(np.ones(k2-1),-1) + np.eye(k2)
I2=np.zeros((k2,k2))
M2=np.zeros((k2,k2))
for j in range(k2):
    I2[:,j] = I[:,k2-j-1]
    M2[:,j] = M[:,k2-j-1]   

Z = np.zeros((k2,k2,nbfilter11,nbfilter12))

for u in range(nbfilter12):
    Z[:,:,u,0] = I
    Z[:,:,u,1] = I2
    Z[:,:,u,2] = M
    Z[:,:,u,3] = M2
    for p in range(4,nbfilter12):
        Z[:,:,u,p]=np.random.randn(k2,k2)*0.3            

In [10]:
# Création d'un modèle similaire à celui enregistré
c2d1_input = keras.Input(shape=(36,36,1))
cnn2d1 = Conv2D(filters = nbfilter11, kernel_size=(k,k), padding='valid', input_shape=(36,36,1),strides=(1,1),weights=[W,np.zeros(nbfilter11)])(c2d1_input)
cnn2d1 = AveragePooling2D(pool_size=(3,3))(cnn2d1)
cnn2d1 = Conv2D(filters = nbfilter12, kernel_size = (k2, k2),strides=(1,1),padding='valid',weights=[Z,np.zeros(nbfilter12)])(cnn2d1)
cnn2d1 = Dropout(0.1)(cnn2d1)
cnn2d1 = Flatten()(cnn2d1)
#cnn2d1 = get_cnn_network(matrixsize11, nbfilter11, matrixsize12, nbfilter12)(c2d1_input)
#training_net1.append(cnn2d1)
c2d2_input = keras.Input(shape=(36,36,1))
cnn2d2 = Conv2D(filters = nbfilter11, kernel_size=(k,k), padding='valid', input_shape=(36,36,1),strides=(1,1),weights=[W,np.zeros(nbfilter11)])(c2d1_input)
cnn2d2 = AveragePooling2D(pool_size=(3,3))(cnn2d2)
cnn2d2 = Conv2D(filters = nbfilter12, kernel_size = (k2, k2),strides=(1,1),padding='valid',weights=[Z,np.zeros(nbfilter12)])(cnn2d2)
cnn2d2 = Dropout(0.1)(cnn2d2)
cnn2d2 = Flatten()(cnn2d2)
#cnn2d2 = get_cnn_network(matrixsize21, nbfilter21, matrixsize22, nbfilter22)(c2d2_input)
#training_net1.append(cnn2d2)
c1d1_input = keras.Input(shape=(36,4))
cnn1d1 = Conv1D(filters = nbfilter1, kernel_size = kernelsize, strides = 1, padding = 'valid', input_shape=(36,4), kernel_initializer = keras.initializers.lecun_uniform(seed=None))(c1d1_input)
cnn1d1 = MaxPooling1D(pool_size=8)(cnn1d1)
cnn1d1 = Dropout(0.2)(cnn1d1)
cnn1d1 = Flatten()(cnn1d1)
#cnn1d1 = get_cnn_network_seq(nbfilter1,kernel_size1)(c1d1_input)
c1d2_input = keras.Input(shape=(36,4))
cnn1d2 = Conv1D(filters = nbfilter1, kernel_size = kernelsize, strides = 1, padding = 'valid', input_shape=(36,4), kernel_initializer = keras.initializers.lecun_uniform(seed=None))(c1d2_input)
cnn1d2 = MaxPooling1D(pool_size=8)(cnn1d2)
cnn1d2 = Dropout(0.2)(cnn1d2)
cnn1d2 = Flatten()(cnn1d2)
#cnn1d2 = get_cnn_network_seq(nbfilter1,kernel_size1)(c1d2_input)
c1d3_input = keras.Input(shape=(36,4))
cnn1d3 = Conv1D(filters = nbfilter1, kernel_size = kernelsize, strides = 1, padding = 'valid', input_shape=(36,4), kernel_initializer = keras.initializers.lecun_uniform(seed=None))(c1d3_input)
cnn1d3 = MaxPooling1D(pool_size=8)(cnn1d3)
cnn1d3 = Dropout(0.2)(cnn1d3)
cnn1d3 = Flatten()(cnn1d3)
#cnn1d3 = get_cnn_network_seq(nbfilter1,kernel_size1)(c1d3_input)
c1d4_input = keras.Input(shape=(36,4))
cnn1d4 = Conv1D(filters = nbfilter1, kernel_size = kernelsize, strides = 1, padding = 'valid', input_shape=(36,4), kernel_initializer = keras.initializers.lecun_uniform(seed=None))(c1d4_input)
cnn1d4 = MaxPooling1D(pool_size=8)(cnn1d4)
cnn1d4 = Dropout(0.2)(cnn1d4)
cnn1d4 = Flatten()(cnn1d4)
#cnn1d4 = get_cnn_network_seq(nbfilter1,kernel_size1)(c1d4_input)
#training_net21.append(cnn1d1)
#training_net21.append(cnn1d2)
#training_net22.append(cnn1d3)
#training_net22.append(cnn1d4)

#model21 = Sequential()
model21 = keras.layers.concatenate([cnn1d1,cnn1d2])
model21 = Dense(int(Dense2/2),kernel_initializer=keras.initializers.lecun_uniform(seed=None))(model21)
model21 = BatchNormalization()(model21)
model21 = Activation('relu')(model21)
model21 = Dropout(0.2)(model21)
#model22 = Sequential()
model22 = keras.layers.concatenate([cnn1d3,cnn1d4])
model22 = Dense(int(Dense2/2),kernel_initializer=keras.initializers.lecun_uniform(seed=None))(model22)
model22 = BatchNormalization()(model22)
model22 = Activation('relu')(model22)
model22 = Dropout(0.2)(model22)
#training_net2.append(model21)
#training_net2.append(model22)
#model2 = Sequential()
model2 = keras.layers.concatenate([model21,model22])
model2 = Dense(Dense2,kernel_initializer=keras.initializers.lecun_uniform(seed=None))(model2)
model2 = BatchNormalization()(model2)
model2 = Activation('relu')(model2)
model2 = Dropout(0.2)(model2)
#model2.add(Dense(Dense2,kernel_initializer=keras.initializers.lecun_uniform(seed=None)))
#model2.add(BatchNormalization())
#model2.add(Activation('relu'))
#model2.add(Dropout(0.2))

#model1 = Sequential()
model1 = keras.layers.concatenate([cnn2d1,cnn2d2])
model1 = Dense(Dense1,kernel_initializer=keras.initializers.lecun_uniform(seed=None))(model1)
model1 = Dropout(0.1)(model1)
model1 = BatchNormalization()(model1)
model1 = Activation('relu')(model1)

#training_net.append(model1)
#training_net.append(model2)
#model = Sequential()
model = keras.layers.concatenate([model1,model2])
model = Dense(Dense3,kernel_initializer=keras.initializers.lecun_uniform(seed=None))(model)
model = BatchNormalization()(model)
model = Activation('relu')(model)
model = Dropout(0.1)(model)
model = Dense(Dense4,kernel_initializer=keras.initializers.lecun_uniform(seed=None))(model)
model = BatchNormalization()(model)
model = Activation('relu')(model)
#model.add(Dropout(0.1))
if Dense5>0:
    model = Dense(Dense5,kernel_initializer=keras.initializers.lecun_uniform(seed=None))(model)
    model = BatchNormalization()(model)
    model = Activation('relu')(model)
model = Dense(2,kernel_initializer=keras.initializers.lecun_uniform(seed=None))(model)
model = BatchNormalization()(model)
interaction_output = Activation('softmax')(model)
#model.add(Activation('softmax'))
interaction = Model(inputs=[c2d1_input,c2d2_input,c1d1_input,c1d2_input,c1d3_input,c1d4_input],outputs=[interaction_output])

#sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
#model.compile(loss='categorical_crossentropy', optimizer='rmsprop')
interaction.compile(loss='categorical_crossentropy', optimizer='rmsprop')

earlystopper = EarlyStopping(monitor='val_loss', patience=2, verbose = verbose1)
print('model training')
#model.fit(training, y, batch_size, epochs, verbose = verbose2, validation_data=(validation, val_y), callbacks=[earlystopper])
interaction.fit(training, y, batch_size, epochs, verbose = verbose2, validation_data=(validation, val_y), callbacks=[earlystopper])
# test   

print('predicting')


#predictions = model.predict_proba(testing)[:,1]
predictions = interaction.predict(testing)[:,1]
print(predictions)
for i,nulll in enumerate(predictions):
    predictions[i] = round(predictions[i])
print(predictions,true_y)
perfs = calculate_performance(len(predictions), predictions, true_y)
print("batch_size : ", batch_size)
print("epochs : ",epochs)
print("acc : ", perfs[0])
print("precision : ", perfs[1])
print("sensitivity : ", perfs[2])
print("specificity : ", perfs[3])
print("MCC : ", perfs[4])

model training
Train on 100 samples, validate on 250 samples
Epoch 1/25
100/100 [==============================] - 1s 7ms/step - loss: 0.8544 - val_loss: 0.7519
Epoch 2/25
100/100 [==============================] - 0s 4ms/step - loss: 0.6518 - val_loss: 1.2051
Epoch 3/25
100/100 [==============================] - 0s 4ms/step - loss: 0.5285 - val_loss: 1.0959
Epoch 00003: early stopping
predicting
[0.2447393  0.34259072 0.08586895 ... 0.28511932 0.26527423 0.28291538]
[0. 0. 0. ... 0. 0. 0.] [0.0 1 0.0 ... 0.0 0.0 0.0]
batch_size :  64
epochs :  25
acc :  0.5024
precision :  0.5
sensitivity :  0.0008038585209003215
specificity :  0.9992038216560509
MCC :  0.00013582040505390896


In [15]:
# Récupération des poids
interaction.load_weights('model_double_final.h5')

In [16]:
# Test des performances (performances supérieures à la réalité car sur
#  des données déjà apprises)

perfs_test(interaction,Data[4],Data[5])

predicting
[0.25861937 0.6298028  0.61370957 ... 0.36930397 0.05166985 0.23346059]
[0. 1. 1. ... 0. 0. 0.] [0.0 0.0 1 ... 0.0 1 1]
acc :  0.758
precision :  0.8095238095238095
sensitivity :  0.6932038834951456
specificity :  0.8268041237113402
MCC :  0.5234308508036729
